In [1]:
#l'importation des modules utilisés
import subprocess
import pandas as pd
import os
import csv
from pathlib import Path
import re
import numpy as np
from datetime import datetime

In [2]:
#les chemins d'accès aux fichiers
root_file="C:/Users/14387"

os.chdir(root_file)
actual_dir=Path.cwd()

project_repo = actual_dir/"Apache_Hive_Bug_Prediction_ML_/"
hive_repo = project_repo.parent/"Apache_Hive/"
os.chdir(hive_repo)

In [3]:
#l'extraction de données sur la base du projet de laboratoire

#dictionnaire de release
hive_versions = {
"2.0.0": "21cf6ff3789ed94bdf61587e2e73fb94b1d9304c", 
"2.0.1": "115d225d8d311e1d1fa42e8f103c214b26c209a2",
"2.1.0": "c168af26de36ecb66e606b040c862a222ee2a190",
"2.1.1": "52f1b2471545a797856e4b9b1ae0a36cb4233c18",
"2.2.0" : "52f1b2471545a797856e4b9b1ae0a36cb4233c18",
"2.3.0": "90149de71eecb3374fcee0b876324a3b84b8a93d",
"2.3.1": "d7d96658c233b100754884985860bf40f97264ee",
"2.3.2": "b2e411e8cad0da0af370e2b3e2d0bf9cb32dfe7f",
"2.3.3": "52f1b2471545a797856e4b9b1ae0a36cb4233c18",
"2.3.4": "353c55e22d0b0f4603053a7ef45afb5b7b244b75",
"2.3.5": "de69a0f9d973822895f3ff0744ee3af768118268",
"2.3.6": "db59ec6ecf57dad72588b7593fb516c3e810b5a2",
"2.3.7": "03efbb3db1bbccfd3548fa7fa3f130be081e1cdd",
"2.3.8": "89073a94354f0cc14ec4ae0a43e05aae29276b4d",
"2.3.9": "01b8fcffcb31cc128e71350d1b25c98b50ee85e5",
"2.3.10": "7950967eae9640fcc0aa22f4b6c7906b34281eac",
"3.0.0": "ed95ee90608a73fe08747b5483f1bc5ff14bfa22",
"3.1.0": "65f02d2f99b990cb28fd6a832fa3425042e60a04",
"3.1.1": "4b40f7d2b35df26af773ec138fdf521231792cdf",
"3.1.2": "01b8fcffcb31cc128e71350d1b25c98b50ee85e5",
"3.1.3": "01b8fcffcb31cc128e71350d1b25c98b50ee85e5",
"4.0.0": "8b287869bd75665cdcf7f59b64389b6e33cfbab8",
"4.0.1": "799b5cfd95c13433ec46a8aaa23e0b2f38f24d24",
}

#recherche par clé dans le dictionnaire
def searchByKey(key):
    keys = list(hive_versions.keys())
    index = keys.index(key)
    index_found=int(index)
    return index_found

#recherche par index dans le dictionnaire
def searchByIndex(index_found):   
    index_found = int(index_found)
    keys = list(hive_versions.keys()) 
    key_index = keys[index_found] 
    value_index = hive_versions[key_index]
    return value_index

def mediaDateCommits(array):
    if len(array)>1:
        # Convert dates to datetime objects
        commit_dates = [datetime.fromisoformat(date.strip()) for date in array]
        
        # Calcular las diferencias entre fechas consecutivas
        time_differences = [(commit_dates[i] - commit_dates[i + 1]).total_seconds() for i in range(len(commit_dates) - 1)]
        
        # Calculate differences between consecutive dates
        median_time_between_changes = np.median(time_differences)
        
        days, remainder = divmod(median_time_between_changes, 86400)
        return int(days)
    else:
        return 0

def gotToHiveRepo():
    root_file="C:/Users/14387"
    os.chdir(root_file)
    actual_dir=Path.cwd()
    
    project_repo = actual_dir/"Apache_Hive_Bug_Prediction_ML_/"
    hive_repo = project_repo.parent/"Apache_Hive/"
    os.chdir(hive_repo)
    actual_dir=Path.cwd()
    print(actual_dir)

#eremoveduplication
def cleanDuplication(array):
    duplicatedCleaned = set(arr.strip() for arr in array) 
    return duplicatedCleaned



def getCommitsByAuthor(authorsArray,hash1,hash2):
    expertArr=[]
    for author in authorsArray:
        commitsDone=commitsAffectedByAutor(author,hash1,hash2)
        expertArr.append(int(commitsDone))

    return expertArr

def calculateAvg(arrayList):
    if len(arrayList) > 1:
        avg = sum(arrayList) / len(arrayList)
        return round(avg)
    else:
        if len(arrayList)==1:
            return arrayList[0]
        else:
            return 0

def calculateMin(arrayList):
    if len(arrayList) > 1:
        minArr = min(arrayList)
        return minArr
    else:
        if len(arrayList)==1:
            return arrayList[0]
        else:
            return 0

       
def commitsAffectedByAutor(authorParm,hash1,hash2):
    author = authorParm
    result = subprocess.run(['git', 'log', '--author', author, f'{hash1}..{hash2}', '--pretty=format:%H'], capture_output=True, text=True)
    commits = result.stdout.strip().split('\n')
    counter_commits = len(commits)
    return counter_commits

In [4]:
repo = actual_dir/"Apache_Hive_Bug_Prediction_ML_"/"files_output_new_metrics/"
os.chdir(repo)
print(repo)



C:\Users\14387\Apache_Hive_Bug_Prediction_ML_\files_output_new_metrics


In [ ]:

version_pattern = re.compile(r'_(\d+\.\d+\.\d+)_')

bug_dfs = []


#write avgDays in V and B4
for file_path in repo.glob("*.csv"):
    os.chdir(repo)
    print(repo)
    df = pd.read_csv(file_path)
    filename = file_path.name  # e.g., 'Hive_3.3.0_Jira_Bug_Data.csv'
    print(filename,"------------------------------------------------")
    
    for file in range(len(df)-1):
        
        filePath=df.loc[file, 'File']
        version=df.loc[file, 'Version']
        print(version)
        dateIndex=int(searchByKey(version))
        #print(dateIndex)
        hash1=""
        hash2=""
        firsthash='243e7c1ac39cb7ac8b65c5bc6988f5cc3162f558'
        if version=="2.0.0":
            hash1=firsthash
        else:
            hash1=str(searchByIndex(dateIndex-1))
         
        hash2=str(searchByIndex(dateIndex))
        #print(firsthash)
        #print(hash2)
        goToRepo=gotToHiveRepo()
        #average number of days between commits avgbtwh---8
        avgbtwhcommand= ['git', 'show','-s','--format=%cd','--date=iso',f'{hash1}..{hash2}', "--",filePath]
        subprocAvgbtwh= subprocess.run(avgbtwhcommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        avgbtwh=0
        outputAvgbtwh = subprocAvgbtwh.stdout.splitlines()
        #print(outputAvgbtwh)
        arrayCommitsDate=outputAvgbtwh
        avgbtwh=mediaDateCommits(arrayCommitsDate)
        goToRepo=gotToHiveRepo()
        #average number of days between commits avgbtwhPlus----9
        avgbtwhPluscommand=  ['git', 'show','-s','--format=%cd','--date=iso',f'{firsthash}..{hash2}', "--",filePath]
        subprocavgbtwhPlus= subprocess.run(avgbtwhPluscommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        outputavgbtwhPlus = subprocavgbtwhPlus.stdout.splitlines()
        #print(outputavgbtwhPlus)
        arrayCommitsDatePlus=outputavgbtwhPlus
        avgbtwhPlus=mediaDateCommits(arrayCommitsDatePlus)


        #git expertise Author

        devCountercommand = ['git', 'log', f'{hash1}..{hash2}', '--pretty=format:%an', "--",filePath]
        devCounter1 = subprocess.run(devCountercommand, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        devcounter1=0
        arrayDev1=[]
        if devCounter1.stdout is not None:
            outputdev1 = devCounter1.stdout.splitlines()
            arrayDevduplication=outputdev1
            print(arrayDevduplication)
            arrayDev1=cleanDuplication(arrayDevduplication)
            print(arrayDev1)
            devcounter1=int(len(arrayDev1))
        print("devCounter----------------------------------",devcounter1)
        #expertise author--------------------------10
        arrAuthorComitsCounter=getCommitsByAuthor(arrayDev1,firsthash,hash2)
        avgExpertiseAuthor=calculateAvg(arrAuthorComitsCounter)
        print(avgExpertiseAuthor)
        
        #min author--------------------------------11
        arrAuthorComitsCounter=getCommitsByAuthor(arrayDev1,firsthash,hash2)
        minExpertiseAuthor=calculateMin(arrAuthorComitsCounter)
        print(minExpertiseAuthor)
        
       
        df.loc[file, 'avgbtwhDays']=avgbtwh
        df.loc[file, 'avgbtwhDaysPlus']=avgbtwhPlus
        df.loc[file, 'expertiseAvg']=avgExpertiseAuthor
        df.loc[file, 'expertiseMin']=minExpertiseAuthor

        
        
        print(filename)
        os.chdir(repo)
      
        df.to_csv(filename, index=False)
        




C:\Users\14387\Apache_Hive_Bug_Prediction_ML_\files_output_new_metrics
File_2.0.0_newMetrics.csv ------------------------------------------------
2.0.0
C:\Users\14387\Apache_Hive
C:\Users\14387\Apache_Hive
['Jesus Camacho Rodriguez', 'Pengcheng Xiong', 'Prasanth Jayachandran', 'Prasanth Jayachandran', 'Ashutosh Chauhan', 'Ashutosh Chauhan', 'Ashutosh Chauhan', 'ctang', 'Daniel Dai', 'Navis Ryu', 'Alan Gates', 'Hari Subramaniyan', 'Sergey Shelukhin', 'Ashutosh Chauhan', 'Thejas Nair', 'Gunther Hagleitner', 'Pengcheng Xiong', 'Pengcheng Xiong', 'Ashutosh Chauhan']
{'Pengcheng Xiong', 'Sergey Shelukhin', 'Gunther Hagleitner', 'Thejas Nair', 'Navis Ryu', 'ctang', 'Prasanth Jayachandran', 'Daniel Dai', 'Ashutosh Chauhan', 'Jesus Camacho Rodriguez', 'Hari Subramaniyan', 'Alan Gates'}
devCounter---------------------------------- 12
79
5
File_2.0.0_newMetrics.csv
2.0.0
C:\Users\14387\Apache_Hive
C:\Users\14387\Apache_Hive
[]
set()
devCounter---------------------------------- 0
0
0
File_2.0.0_n

Exception in thread Thread-54475 (_readerthread):
Traceback (most recent call last):
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1075, in _bootstrap_inner
    self.run()
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\threading.py", line 1012, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "C:\Users\14387\anaconda3\envs\MasterSoftwareEngineering\Lib\encodings\cp1252.py", line 23, in decode
    return codecs.charmap_decode(input,self.errors,decoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeDecodeError: 'charmap' codec can't decode byte 0x8f in

devCounter---------------------------------- 0
0
0
File_3.1.2_newMetrics.csv
3.1.2
C:\Users\14387\Apache_Hive
C:\Users\14387\Apache_Hive
['Zoltan Haindrich', 'Denys Kuzmenko', 'Denys Kuzmenko', 'dengzh', 'Vihang Karajgaonkar', 'LÃ¡szlÃ³ PintÃ©r', 'Ayush Saxena', 'Dataproc Metastore', 'saihemanth', 'Ashish Kumar Sharma', 'Zoltan Haindrich', 'Zoltan Haindrich', 'YU-WEN LAI', 'Naveen Gangam', 'Aasha Medhi', 'David Lavati', 'Jesus Camacho Rodriguez', 'David Lavati', 'Mahesh Kumar Behera', 'Jesus Camacho Rodriguez', 'Ashutosh Bapat', 'Miklos Gergely', 'Sankar Hariappan', 'Miklos Gergely', 'Sankar Hariappan', 'Ashutosh Bapat', 'Sankar Hariappan', 'Sankar Hariappan', 'Mahesh Kumar Behera']
{'David Lavati', 'Denys Kuzmenko', 'Ashutosh Bapat', 'LÃ¡szlÃ³ PintÃ©r', 'Vihang Karajgaonkar', 'Ayush Saxena', 'Ashish Kumar Sharma', 'Aasha Medhi', 'Mahesh Kumar Behera', 'Naveen Gangam', 'saihemanth', 'Jesus Camacho Rodriguez', 'Miklos Gergely', 'Dataproc Metastore', 'Sankar Hariappan', 'YU-WEN LAI', 'de

In [ ]:
#write devaverage

In [ ]:
#search on message code line commented